# Project nº1 - MapReduce

Use MapReduce to create indexes for answering the following queries:

## 1.How many trips were started in each year present in the data set?

Output is expected to have two columns: (year, #trips). In the example below the values are not
real and you may get different ones.

## Mapper



In [1]:
%%file mapper_ex1.py
#!/usr/bin/env python
import sys
import string

for line in sys.stdin:
    line=line.strip()
    #guardar a data de inicio de viagem na variavel date
    date = line.split(";")[2]
    
    date_1 = date.split()[0]
    #a partir da data obter o ano em que ocorreu cada viagem
    year = date_1.split("/")[2]
    
    print('%s\t1'% (year))


Writing mapper_ex1.py


## Reducer




In [22]:
%%file reducer_ex1.py
#!/usr/bin/python
import sys

last_year = ''
total = 0

for line in sys.stdin:
    line = line.strip()
    year ,count = line.split('\t',1)
    count = int(count)
    
    if last_year != year:
        if last_year != '':
            print("%s\t%d" % (last_year, total))
        last_year = year
        total = count
    else:
        total += count

print("%s\t%d" % (last_year, total))

Overwriting reducer_ex1.py


### Hadoop standalone mode execution

In [23]:
rm -rf results_ex1_mapreduce

#### Submitting the job



In [24]:
%%time
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_ex1.py,reducer_ex1.py -mapper mapper_ex1.py -reducer reducer_ex1.py -input Taxi_Trips_151MB.csv -output results_ex1_mapreduce

2020-11-02 16:11:40,234 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2020-11-02 16:11:40,315 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2020-11-02 16:11:40,316 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2020-11-02 16:11:40,335 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2020-11-02 16:11:40,530 INFO mapred.FileInputFormat: Total input files to process : 1
2020-11-02 16:11:40,562 INFO mapreduce.JobSubmitter: number of splits:5
2020-11-02 16:11:40,737 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1955230263_0001
2020-11-02 16:11:40,737 INFO mapreduce.JobSubmitter: Executing with tokens: []
2020-11-02 16:11:40,991 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jovyan/work/mapper_ex1.py as file:/tmp/hadoop-jovyan/mapred/local/job_local1955230263_0001_efc26dfa-3765-40d7-a955-164eae5a1784/mapper_ex1.py
2020-11-02 16:11:41,005 INFO mapred.Loca

#### Checking the results


In [25]:
!cat results_ex1_mapreduce/part-*

2013	54409
2014	74753
2015	64761
2016	63628
2017	50006
2018	41567
2019	32797
2020	6829


## 2.For each of the 24 hours of the day, how many taxi trips there were, what was their average trip miles and trip total cost? Non-integer values should be printed with two decimal places.

Output is expected to have four columns (time_slot, #trips, avg_trip_miles, avg_trip_total). In
the example below the values are not real and you may get different ones.

## Mapper



In [26]:
%%file mapper_ex2.py
#!/usr/bin/env python
import sys
import string

for line in sys.stdin:
    line=line.strip() 
    
    info = line.split(";")
    date = info[2].split()
    date_2 = date[1].split(":")[0]
    miles = info[5].replace(',','')
    cost = info[14].replace(',','')
    
    if miles == '':
        miles = 0
    if cost == '':
        cost = 0
    
    print("%s %s\t%s %s" % (date_2, date[2], miles, cost ))
    

Overwriting mapper_ex2.py


## Reducer




In [7]:
%%file reducer_ex2.py
#!/usr/bin/python
import sys

last_hour = ''
trips = 0


for line in sys.stdin:
    line = line.strip()
    hour , info = line.split('\t',1)
    miles , cost = info.split()
    miles = float(miles)
    cost = float(cost)
    
    
    if last_hour != hour:
        if last_hour == '':
            last_hour = hour
        if trips!=0:
            avg_miles = miles_total / trips
            avg_cost = cost_total / trips
            print("%s\t%d\t%0.2f\t%0.2f" % (last_hour, trips, avg_miles, avg_cost))
            last_hour = hour
            miles_total = miles
            cost_total = cost
            trips = 1
        else:
            last_hour = hour
            miles_total = miles
            cost_total = cost
            trips = 1          
    
    else:
        trips += 1
        miles_total += miles
        cost_total += cost
        
avg_miles = miles_total / trips
avg_cost = cost_total / trips
print("%s\t%d\t%0.2f\t%0.2f" % (last_hour, trips, avg_miles, avg_cost))


Writing reducer_ex2.py


### Hadoop standalone mode execution

In [8]:
rm -rf results_ex2_mapreduce

#### Submitting the job



In [9]:
%%time
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper_ex2.py,reducer_ex2.py -mapper mapper_ex2.py -reducer reducer_ex2.py -input Taxi_Trips_151MB.csv -output results_ex2_mapreduce

2020-11-02 16:10:36,636 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2020-11-02 16:10:36,723 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2020-11-02 16:10:36,723 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2020-11-02 16:10:36,741 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2020-11-02 16:10:36,923 INFO mapred.FileInputFormat: Total input files to process : 1
2020-11-02 16:10:36,938 INFO mapreduce.JobSubmitter: number of splits:5
2020-11-02 16:10:37,107 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local711160793_0001
2020-11-02 16:10:37,107 INFO mapreduce.JobSubmitter: Executing with tokens: []
2020-11-02 16:10:37,370 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jovyan/work/mapper_ex2.py as file:/tmp/hadoop-jovyan/mapred/local/job_local711160793_0001_1b54b00b-4af2-4269-8cd4-a99ef7923f79/mapper_ex2.py
2020-11-02 16:10:37,380 INFO mapred.LocalD

#### Checking the results


In [10]:
!cat results_ex2_mapreduce/part-*

01 AM	11166	2.46	13.73
01 PM	20181	3.38	15.79
02 AM	8832	2.35	12.71
02 PM	20039	3.56	16.10
03 AM	6594	2.48	12.96
03 PM	20708	3.59	17.38
04 AM	4604	3.80	15.19
04 PM	21714	3.34	16.87
05 AM	4087	5.94	20.94
05 PM	23639	3.09	15.20
06 AM	5629	5.57	19.93
06 PM	25446	2.94	14.62
07 AM	10145	3.98	17.09
07 PM	25402	2.94	15.26
08 AM	15695	2.98	13.96
08 PM	22222	3.18	15.99
09 AM	18248	3.23	14.47
09 PM	19786	3.40	15.95
10 AM	17777	3.30	15.34
10 PM	18492	3.11	15.29
11 AM	18622	3.56	16.48
11 PM	16303	3.07	15.00
12 AM	13544	2.83	14.45
12 PM	19875	3.38	16.04


## 3.For each of the 24 hours of the day, which are the (up to) 5 most popular routes (pairs pickup/dropoff regions) according to the the total number of taxi trips? Also report and the average fare (total trip cost). Non-integer values should be printed with two decimal places.

Output is expected to have four columns (pickup_location, #trips, avg_trip_miles,
avg_trip_total). In the example below the values are not real and you may get different ones.

# 1ºMapper



In [21]:
%%file mapper1_3ex.py
#!/usr/bin/env python
import sys
import string

for line in sys.stdin:
    line=line.strip() 
    
    info = line.split(";")
    date = info[2].split()
    date_2 = date[1].split(":")[0]
    
    pickup=info[6]
    dropoff=info[7]
    
    miles = info[5].replace(',','')
    cost = info[14].replace(',','')
    
    if not pickup:
        continue
    if not dropoff:
        continue
    if not miles :
        continue
    if not cost:
        continue
    
    print("%s %s %s %s\t%s %s" % (date_2, date[2],pickup,dropoff, miles, cost))
    


Overwriting mapper1_3ex.py


## 1ºReducer




In [12]:
%%file reducer1_3ex.py
#!/usr/bin/python
import sys

last_key = ''
trips = 0
top_list={}

for line in sys.stdin:
    line = line.strip()
    key , info = line.split('\t',1)
    miles , cost = info.split()
    miles = float(miles)
    cost = float(cost)
    hour=key[0:5]
    key=key[6:]
    
    if last_key != key:
        if last_key == '':
            last_key = key
            last_hour=hour
        if trips!=0:
            avg_miles = miles_total / trips
            avg_cost = cost_total / trips
            print("%s\t%05d\t%s\t%s\t%s"%(hour,trips,key,avg_miles,avg_cost))
                        
            last_key = key
            miles_total = miles
            cost_total = cost
            trips = 1
        else:
            last_key = key
            miles_total = miles
            cost_total = cost
            trips = 1          
    else:
        trips += 1
        miles_total += miles
        cost_total += cost
print("%s\t%05d\t%s\t%s\t%s"%(hour,trips,key,avg_miles,avg_cost))




Writing reducer1_3ex.py


### Hadoop standalone mode execution

In [13]:
rm -rf results_ex3pt1_mapreduce

#### Submitting the job


In [14]:
%%time
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar -files mapper1_3ex.py,reducer1_3ex.py -mapper mapper1_3ex.py -reducer reducer1_3ex.py -input Taxi_Trips_151MB.csv -output results_ex3pt1_mapreduce

2020-11-02 16:10:53,200 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2020-11-02 16:10:53,295 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2020-11-02 16:10:53,295 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2020-11-02 16:10:53,322 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2020-11-02 16:10:53,500 INFO mapred.FileInputFormat: Total input files to process : 1
2020-11-02 16:10:53,526 INFO mapreduce.JobSubmitter: number of splits:5
2020-11-02 16:10:53,698 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1974144350_0001
2020-11-02 16:10:53,698 INFO mapreduce.JobSubmitter: Executing with tokens: []
2020-11-02 16:10:53,969 INFO mapred.LocalDistributedCacheManager: Localized file:/home/jovyan/work/mapper1_3ex.py as file:/tmp/hadoop-jovyan/mapred/local/job_local1974144350_0001_64fab87b-9d73-4bb0-b03e-e03fbe049b3f/mapper1_3ex.py
2020-11-02 16:10:53,981 INFO mapred.Lo

#### Checking the results


In [15]:
!cat results_ex3pt1_mapreduce/part-*

01 AM	00001	17031020500 17031020500	0.0	13.45
01 AM	00001	17031020901 17031221500	0.0	3.25
01 AM	00001	17031030102 17031832000	5.2	17.5
01 AM	00001	17031030200 17031010400	0.2	13.85
01 AM	00001	17031030200 17031031400	3.7	13.95
01 AM	00001	17031030500 17031031000	4.0	13.65
01 AM	00001	17031030603 17031842300	1.1	5.65
01 AM	00001	17031030800 17031031300	0.5	19.65
01 AM	00001	17031030800 17031040201	0.9	6.05
01 AM	00001	17031030800 17031050100	0.0	7.25
01 AM	00001	17031030800 17031081401	0.0	11.45
01 AM	00001	17031030800 17031830700	0.4	17.05
01 AM	00001	17031030900 17031031100	1.9	7.65
01 AM	00001	17031030900 17031062000	0.5	7.25
01 AM	00001	17031030900 17031062100	4.1	14.25
01 AM	00002	17031030900 17031063302	1.35	12.8
01 AM	00001	17031030900 17031832000	0.0	14.85
01 AM	00001	17031031000 17031030500	0.1	12.6
01 AM	00001	17031031000 17031031000	0.9	4.85
01 AM	00001	17031031000 17031031100	0.0	3.25
01 AM	00001	17031031000 17031031200	0.0	5.85
01 AM	00001	17031031000 17031060300	0.7	5.84


## 2ºMapper




In [16]:
%%file mapper2_3ex.py
#!/usr/bin/env python
import sys
import string

for line in sys.stdin:
    line=line.strip() 
    
    
    print(line)
    


Writing mapper2_3ex.py


## 2ºReducer




In [17]:
%%file reducer2_3ex.py
#!/usr/bin/python
import sys

last_hour = ''
trips = 0
top_list={}
aux=0
for line in sys.stdin:
    line = line.strip()
    
    info = line.split("\t")
    trips = info[1]
    key=info[2]    
    avg_miles=float(info[3])
    avg_cost=float(info[4])
    hour=info[0]
    if last_hour != hour:
        if last_hour == '':
            last_hour = hour
        last_hour = hour
        aux=0
    if(aux<5):
        print("%s %s\t %d %0.2f %0.2f" % (last_hour,key, int(trips),avg_miles,avg_cost))
        aux+=1
    
    
    
#cat Taxi_Trips_151MB.csv |python mapper1_3ex.py|python reducer1_3ex.py|python mapper2_3ex.py|sort -k1 -r|python reducer2_3ex.py

Writing reducer2_3ex.py


### Hadoop standalone mode execution

In [18]:
rm -rf results_ex3pt2_mapreduce

#### Submitting the job



In [19]:
%%time
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar \
-D stream.map.output.field.separator='\t' \
-D stream.num.output.key.fields=5 \
-D map.output.key.field.separator='\t' \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator\
-D mapred.text.key.comparator.options=-k1,1r\
-files mapper2_3ex.py,reducer2_3ex.py -mapper mapper2_3ex.py -reducer reducer2_3ex.py -input results_ex3pt1_mapreduce -output results_ex3pt2_mapreduce

2020-11-02 16:11:06,314 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2020-11-02 16:11:06,394 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2020-11-02 16:11:06,394 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2020-11-02 16:11:06,417 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2020-11-02 16:11:06,604 INFO mapred.FileInputFormat: Total input files to process : 1
2020-11-02 16:11:06,619 INFO mapreduce.JobSubmitter: number of splits:1
2020-11-02 16:11:06,658 INFO Configuration.deprecation: map.output.key.field.separator is deprecated. Instead, use mapreduce.map.output.key.field.separator
2020-11-02 16:11:06,658 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
2020-11-02 16:11:06,659 INFO Configuration.deprecation: mapred.output.key.comparator.class is deprecated. Instead, use mapreduce.job.out

#### Checking the results


In [20]:
!cat results_ex3pt2_mapreduce/part-*

12 PM 17031839100 17031841000	 429 0.81 7.13
12 PM 17031839100 17031320400	 250 0.76 7.59
12 PM 17031320100 17031841000	 223 0.73 7.58
12 PM 17031839100 17031081800	 199 0.69 7.61
12 PM 17031081500 17031841000	 165 1.05 7.92
12 AM 17031081700 17031081800	 96 0.37 6.49
12 AM 17031081700 17031081900	 58 0.56 6.88
12 AM 17031081800 17031081800	 50 0.37 8.35
12 AM 17031081700 17031320400	 50 0.73 7.39
12 AM 17031839100 17031841000	 41 0.62 7.73
11 PM 17031081700 17031081800	 75 0.50 7.44
11 PM 17031839100 17031081800	 70 0.71 7.31
11 PM 17031839100 17031320400	 64 0.69 7.08
11 PM 17031081700 17031841000	 58 0.63 7.03
11 PM 17031081700 17031140702	 55 0.63 6.79
11 AM 17031839100 17031840300	 447 0.66 6.98
11 AM 17031839100 17031320400	 241 0.76 7.91
11 AM 17031320100 17031841000	 214 0.85 7.53
11 AM 17031839100 17031081800	 160 0.83 7.43
11 AM 17031839100 17031081600	 154 1.02 7.89
10 PM 17031839100 17031841000	 97 0.65 6.63
10 PM 17031839100 17031320400	 86 0.81 7.26
10 PM 17031980100 1703

## 4.For each of the month of every year, which are the top 5 taxi drivers according to the the total number of taxi trips? Also report the taxi driver company's and the total trips of each taxi driver.

Output is expected to have two columns (month_year) and a top 5 list of (taxi->id->nºtrips->company). 
e.g.: output: ano_mes ,{ id->taxista->nºtrips, id->taxista->nºtrips,...} (top 5)

## Mapper



In [27]:
%%file mapper_4ex.py
#!/usr/bin/env python
import sys
import string

for line in sys.stdin:
    line=line.strip() 
    
    info = line.split(";")
    
    mes_ano= info[2][6:-11]+"_"+info[2][:-20]
    
    taxi_id=info[1]
    company=info[16]
    
    if not company:
        company="Independente"
        
    if not taxi_id:
        continue
        
    if not mes_ano:
        continue
        
    print("%s\t %s;%s" % (mes_ano,taxi_id,company))


Writing mapper_4ex.py


## Reducer




In [28]:
%%file reducer_4ex.py
#!/usr/bin/python
import sys

last_taxi_id = ''
trips = 0
top5=[]

for line in sys.stdin:
    line = line.strip()
    date,info   = line.split('\t',1)
    taxi_id,company=info.split(";")
    
    if last_taxi_id != taxi_id:
        if last_taxi_id == '':
            last_taxi_id = taxi_id
            last_date=date
        if trips!=0:
            if(len(top5)<5 ):
                top5.append([trips,last_taxi_id,company])
            else:
                length=len(top5)
                top5.sort()
                for i in range(length):
                    if trips>int(top5[i][0]):
                        top5[i]=[trips,last_taxi_id,company]
                        break
            if(date!=last_date):
                length=len(top5)
                top5.sort(reverse=True)
                print("%s{"%(last_date))
                aux=0
                for j in range(length):
                    aux+=1
                    if(aux==5):
                        print("%s->%s->%s}" % (top5[j][1],top5[j][0],top5[j][2]))
                    else:
                        print("%s->%s->%s," % (top5[j][1],top5[j][0],top5[j][2]))
                top5=[]
                last_date=date
            last_taxi_id = taxi_id
            trips = 1
        else:
            last_taxi_id = taxi_id
            trips = 1          
    else:
        trips += 1
        

length=len(top5)
top5.sort(reverse=True)
print("%s{"%(date))
aux=0
for j in range(length):
    aux+=1
    if(aux==5):
        print("%s->%s->%s}" % (top5[j][1],top5[j][0],top5[j][2]))
    else:
        print("%s->%s->%s," % (top5[j][1],top5[j][0],top5[j][2]))

        
#cat Taxi_Trips_151MB.csv |python mapper_4ex.py|sort -k1|python reducer_4ex.py




Writing reducer_4ex.py


### Hadoop standalone mode execution

In [29]:
rm -rf results_top5_ex4

#### Submitting the job


In [30]:
%%time
!hadoop jar /opt/hadoop-3.2.0/share/hadoop/tools/lib/hadoop-*streaming*.jar \
-D stream.map.output.field.separator='\t' \
-D stream.num.output.key.fields=5 \
-D map.output.key.field.separator='\t' \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator\
-D mapred.text.key.comparator.options=-k1,1\
-files mapper_4ex.py,reducer_4ex.py -mapper mapper_4ex.py -reducer reducer_4ex.py -input Taxi_Trips_151MB.csv -output results_top5_ex4

2020-11-02 16:12:22,256 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2020-11-02 16:12:22,348 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2020-11-02 16:12:22,348 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2020-11-02 16:12:22,368 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2020-11-02 16:12:22,560 INFO mapred.FileInputFormat: Total input files to process : 1
2020-11-02 16:12:22,586 INFO mapreduce.JobSubmitter: number of splits:5
2020-11-02 16:12:22,631 INFO Configuration.deprecation: map.output.key.field.separator is deprecated. Instead, use mapreduce.map.output.key.field.separator
2020-11-02 16:12:22,632 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
2020-11-02 16:12:22,632 INFO Configuration.deprecation: mapred.output.key.comparator.class is deprecated. Instead, use mapreduce.job.out

#### Checking the results


In [31]:
!cat results_top5_ex4/part-*

2013 _01{	
 f9f12d79733b1fa7934f8d9bd17ca1927f3c99ded1640bbd1c77ef4f0e8a5992897a445315545bedf550405a9c5bc5a9f4b8b03c7d06a1f36050a32c9733164d->6->Blue Ribbon Taxi Association Inc.,	
 c87cc2cd196dc244bb69afcef0ef38d346de12a2657ad7bf400e66e1e50227a6729fc5e2ff90d2d746753569c05d3b0c0dc01f9c1857ddab90e582f931245dce->6->Northwest Management LLC,	
 b07a41c369d99e7e7638e698063efe74c8cd8c25e3e9e88418e4e06ebd947953aae331941132ca50b045c38c7b1e347867ff2eaa181a665e478a66c3ebe74239->6->Taxi Affiliation Services,	
 a9761eac91e87c33b580e7b120c5cc0bde3b3cbe65a70a75a3754e82ff40d74b5b165d734ed6ec8e1c66235bde8564f3c2efd051a393a5daea12ee02bab2ae95->6->Independente,	
 0bcd12ac66d58ecb71d861d9703172ea50786361f34a48db43568b7a7386f8a8d397a75485489bf697eb75180c0f77590cf961e6a7e6145f808efded7dea0383->6->Independente}	
2013 _02{	
 14275cab8bb64007379de40be92944817231f63047033992a1964ce85a4a5405085b87a804736c24ecd8a334baa315255c066e271d7f73dfb756a19a24a44d25->9->Independente,	
 ad33dffdd6cd00795ea1a00a6a6db1a384820